# Access cloud data examples tutorial
 
Credits: Tutorial development
* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org)    - Farallon Institute, USA
* [Dr. Ryan Abernathey](mailto:rpa@ldeo.columbia.edu) - LDEO
* [Henri Drake](mailto:hdrake@mit.edu) - MIT 

Data on the cloud can be stored in many different formats.  Here we will demonstrate some ways to get at the CMIP6, MUR SST, and AVISO data.  


First start by importing libraries


In [ ]:
import warnings
import xarray as xr
import fsspec
from matplotlib import pyplot as plt
import numpy as np
import cartopy
import intake

warnings.simplefilter('ignore') # filter some warning messages
xr.set_options(display_style="html")  #display dataset nicely 
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

## Read MUR SST from AWS public dataset program on an s3 bucket

The MUR SSTs have been reformated into a cloud optimized zarr format.  

Reading the entire global, 1 km, daily, 18 years of data is as simple as pointing to the directory the files are stored in.

This Pangeo binder is running on Google Cloud and will run slower for this data because it is on AWS.  There is a Pangeo binder for AWS as well, but for this tutorial we are using the Google Cloud.
Wall time GC: 1min43sec
Wall time AWS: 33sec

In [ ]:
%%time
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)
ds_sst

### We aren't going to visualize the MUR SST data because it can't take a while since we are on a different cloud

AWS Wall time = 1 min 16 sec to read 1 day of data

AWS Wall time = 1 min 39 sec to read entire 18 years of data at 1 point

In [4]:
#%%time
#sst_day = ds_sst['analysed_sst'].sel(time='2015-10-01T09').load()
#sst_day.sel(lat=slice(20,60),lon=slice(-175,-110)).plot()

In [ ]:
#%%time
#sst_timeseries = ds_sst['analysed_sst'].sel(lat=50,lon=-140,method='nearest').load()
#sst_timeseries.plot()

## Read in AVISO sea surface height data using intake from the Pangeo datastore on Google Cloud

In [ ]:
%%time
cat_pangeo = intake.Catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean.yaml")
ds_aviso = cat_pangeo["sea_surface_height"].to_dask()
ds_aviso

In [ ]:
%%time
ds_aviso['adt'].sel(time='2015-01-01').plot()

## Read CMIP6 data from Google Cloud using intake

The CMIP6 data is a huge collection of different experiements.  Access to these data uses the intake library which you then use the catalog to select specific variables, experiments, or activities.

In [ ]:
%%time
col = intake.open_esm_datastore("https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json")
col

## Search the collection for historical, monthly, air temperature, for one realization

In [ ]:
cat_cmip = col.search(experiment_id=['ssp585','historical'],  # pick the `historical` forcing experiment
                 table_id='Amon',             # choose to look at atmospheric variables (A) saved at monthly resolution (mon)
                 variable_id='tas',           # choose to look at near-surface air temperature (tas) as our variable
                 member_id = 'r1i1p1f1')      # arbitrarily pick one realization for each model (i.e. just one set of initial conditions)


## Convert data catalog into a dictionary of xarray datasets

In [ ]:
dset_dict = cat_cmip.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False})
time_slice = slice('1850','2100') # specific years that bracket our period of interest

ds_dict = {}
gmst_dict = {}
for name, ds in dset_dict.items():
    # rename spatial dimensions if necessary
    if ('longitude' in ds.dims) and ('latitude' in ds.dims):
        ds = ds.rename({'longitude':'lon', 'latitude': 'lat'}) # some models labelled dimensions differently...
        
    ds = xr.decode_cf(ds) # temporary hack, not sure why I need this but has to do with calendar-aware metadata on the time variable
    ds = ds.sel(time=time_slice) # subset the data for the time period of interest
    
    # drop redundant variables (like "height: 2m")
    for coord in ds.coords:
        if coord not in ['lat','lon','time']:
            ds = ds.drop(coord)
    
    # Add near-surface air temperature to dictionary
    ds_dict[name] = ds

## Look at how air temperatures have changed 

In [ ]:
ssp_ds = ds_dict[list(ds_dict.keys())[0]]
now = ssp_ds.sel(time=slice('1995-01-01','1999-12-31')).mean('time')
then = ssp_ds.sel(time=slice('1850-01-01','1950-12-31')).mean('time')
temperature_change=(now-then)['tas']

## Plot the change in temperature

In [ ]:
ortho = ccrs.Orthographic(-90, 20) # define target coordinate frame
geo = ccrs.PlateCarree() # define origin coordinate frame

plt.figure(figsize=(9,7))
ax = plt.subplot(1, 1, 1, projection=ortho)

q = temperature_change.plot(ax=ax, transform = geo,cmap='OrRd',vmin=0,vmax=2) # plot a colormap in transformed coordinates

ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle='-')
plt.title('Patterns of global warming over the Americas',fontsize=16, ha='center');
#plt.savefig('../figures/ssp585_warming_patterns.png',dpi=100,bbox_inches='tight')